In [1]:
!pwd

'pwd' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [2]:
import pandas as pd

# Carregar o arquivo Excel enviado
file_path = 'Pixsee Planet subscription .xlsx'
df = pd.read_excel(file_path)

# Verificar as primeiras linhas do DataFrame para entender a estrutura
df.head()


,Subscription plan,Standard,Gold Star
0,Cost,Complimentary,Approx.$8USD/mo
1,Cloud capacity,Capacity limit 1G,Capacity limit 30G
2,Digital Keepsakes,NaN,NaN
3,Manual Capture (5MP) and Record ⓘ,✓,✓
4,Camera Timer ⓘ,✓,✓


In [3]:
import json
# Atualizar o código para garantir que todas as strings sejam convertidas para letras maiúsculas no JSON
structured_data_with_categories_lowercase = []

# Adicionar detalhes gerais (primeira linha) com letras maiúsculas
for plan, cost in zip(df.columns[1:], df.iloc[0, 1:]):
    structured_data_with_categories_lowercase.append({
        "id": f"{plan.lower()}_general",
        "type": "general_info",
        "plan": plan.lower(),
        "category": "subscription plan",
        "feature": "cost",
        "details": str(cost).lower() if isinstance(cost, str) else cost,
        "subgroup": None
    })

# Processar cada linha para associar com subgrupos e categorizar recursos com letras maiúsculas
current_subgroup = None
for index, row in df.iterrows():
    if index <= 1:
        # Pular as primeiras duas linhas já processadas
        continue

    feature = str(row[df.columns[0]]).lower()  # Converter o recurso para maiúsculas

    # Verificar se a linha representa um novo subgrupo
    if pd.isna(row[df.columns[1]]) and pd.isna(row[df.columns[2]]):
        current_subgroup = feature  # Atualizar o subgrupo atual
        continue

    # Processar os detalhes dos recursos sob o subgrupo atual
    for plan in df.columns[1:]:
        if pd.notna(row[plan]):
            structured_data_with_categories_lowercase.append({
                "id": f"{plan.lower()}_{index}",
                "type": "feature",
                "plan": plan.lower(),
                "category": "feature",
                "feature": feature,
                "details": str(row[plan]).lower() if isinstance(row[plan], str) else row[plan],
                "subgroup": current_subgroup
            })

# Salvar os dados estruturados com categorias em letras maiúsculas em um arquivo JSON
output_path_with_categories_lowercase = 'structured_subscription_data_with_categories_lowercase.json'
with open(output_path_with_categories_lowercase, 'w') as f:
    json.dump(structured_data_with_categories_lowercase, f, indent=4)


